In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['PKD']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('protein kinase d', 515),
 ('polycystic kidney disease', 239),
 ('paroxysmal kinesigenic dyskinesia', 25),
 ('proliferative kidney disease', 14),
 ('pyruvate kinase deficiency', 5),
 ('protein kinase d1', 3),
 ('pseudokinase domain', 3),
 ('pseudoknot domain', 3)]

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'CHEBI:CHEBI:17053': 'L-aspartic acid',
 'MESH:C537180': 'Familial paroxysmal dystonia',
 'HGNC:6647': 'LMOD1',
 'HGNC:9407': 'PRKD1'}

In [12]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

KeyboardInterrupt: 

In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'paroxysmal kinesigenic dyskinesia': 'MESH:C537180',
  'polycystic kidney disease': 'NCIT:C75464',
  'proliferative kidney disease': 'ungrounded',
  'protein kinase d': 'HGNC:9407',
  'protein kinase d1': 'HGNC:9407',
  'pseudokinase domain': 'ungrounded',
  'pseudoknot domain': 'ungrounded',
  'pyruvate kinase deficiency': 'ungrounded'},
 {'MESH:C537180': 'Familial paroxysmal dystonia',
  'NCIT:C75464': 'Polycystic Kidney Disease',
  'HGNC:9407': 'PRKD1'},
 ['HGNC:9407', 'MESH:C537180', 'NCIT:C75464']]

In [13]:
grounding_map, names, pos_labels = [{'paroxysmal kinesigenic dyskinesia': 'MESH:C537180',
  'polycystic kidney disease': 'NCIT:C75464',
  'proliferative kidney disease': 'ungrounded',
  'protein kinase d': 'HGNC:9407',
  'protein kinase d1': 'HGNC:9407',
  'pseudokinase domain': 'ungrounded',
  'pseudoknot domain': 'ungrounded',
  'pyruvate kinase deficiency': 'ungrounded'},
 {'MESH:C537180': 'Familial paroxysmal dystonia',
  'NCIT:C75464': 'Polycystic Kidney Disease',
  'HGNC:9407': 'PRKD1'},
 ['HGNC:9407', 'MESH:C537180', 'NCIT:C75464']]

In [14]:
excluded_longforms = []

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 04:19:45] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 04:19:54] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9676723372544084 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'NCIT:C75464': 185,
  'HGNC:9407': 408,
  'ungrounded': 19,
  'MESH:C537180': 18},
 'f1': {'mean': 0.967672, 'std': 0.016271},
 'precision': {'mean': 0.965325, 'std': 0.016298},
 'recall': {'mean': 0.972144, 'std': 0.01522},
 'HGNC:9407': {'f1': {'mean': 0.977302, 'std': 0.010221},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.955808, 'std': 0.019627}},
 'MESH:C537180': {'f1': {'mean': 0.971429, 'std': 0.057143},
  'pr': {'mean': 0.95, 'std': 0.1},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'NCIT:C75464': {'f1': {'mean': 0.946377, 'std': 0.027863},
  'pr': {'mean': 0.913514, 'std': 0.043243},
  'rc': {'mean': 0.98283, 'std': 0.023309}},
 'ungrounded': {'f1': {'mean': 0.83619, 'std': 0.10748},
  'pr': {'mean': 0.733333, 'std': 0.161589},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [22]:
disamb.dump(model_name, results_path)

In [23]:
print(disamb.info())

Disambiguation model for PKD

Produces the disambiguations:
	Familial paroxysmal dystonia*	MESH:C537180
	PRKD1*	HGNC:9407
	Polycystic Kidney Disease*	NCIT:C75464

Class level metrics:
--------------------
Grounding                   	Count	F1     
                       PRKD1*	408	 0.9773
   Polycystic Kidney Disease*	185	0.94638
                  Ungrounded	 19	0.83619
Familial paroxysmal dystonia*	 18	0.97143

Weighted Metrics:
-----------------
	F1 score:	0.96767
	Precision:	0.96532
	Recall:		0.97214

* Positive labels
See Docstring for explanation



In [24]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1